In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path

src_path = Path('/u/trappist-r0/govorcin/01_OPERA/TROPO/opera_tropo/src')
sys.path.append(str(src_path))

In [3]:
import opera_tropo

In [6]:
import RAiDER
import logging

logger = logging.getLogger('RAiDER')

In [8]:
logger.handlers

[]

In [6]:
from opera_tropo import product_info
from opera_tropo.utils import get_hres_datetime
from opera_tropo.config import runconfig, pge_runconfig

# Product info
metadata of the output product

In [7]:
tropo = product_info.TROPO_PRODUCTS

In [8]:
for item, key in tropo.__dict__.items():
    print(item,'\n',50*'#')
    for i, ik in key.__dict__.items():
        if item == 'coords':
            print(i)
            for ix, ik in ik.__dict__.items():
                print(' ', ix,':', ik)
        else:
            print(i,':', ik)

    print('\n')

wet_delay 
 ##################################################
name : wet_delay
long_name : Zenith Wet Delay
description : One-way Zenith Wet Delay.
fillvalue : nan
missing_value : nan
dtype : <class 'numpy.float32'>
attrs : {'units': 'meters', 'grid_mapping': 'spatial_ref'}
keep_bits : 10


hydrostatic_delay 
 ##################################################
name : hydrostatic_delay
long_name : Zenith Hydrostatic Delay
description : One-way Zenith Wet Delay.
fillvalue : nan
missing_value : nan
dtype : <class 'numpy.float32'>
attrs : {'units': 'meters', 'grid_mapping': 'spatial_ref'}
keep_bits : 12


coords 
 ##################################################
latitude
  axis : Y
  units : degrees_north
  standard_name : latitude
  long_name : Latitude
  description : Angular distance of a point north or south of the equator.
  encoding : {}
longitude
  axis : X
  units : degrees_east
  standard_name : longitude
  long_name : Longitude
  description : Angular distance of a point east 

# Configuration 

In [45]:
tropo_workflow = runconfig.TropoWorkflow
tropo_workflow = tropo_workflow.from_yaml('/u/aurora-r0/govorcin/01_OPERA/TROPO/interface/tropo/src/opera_tropo/config/default/runconfig.yaml')

In [46]:
# Print config defaults
tropo_workflow.print_yaml_schema()

input_options:
  # Path to the input HRES model hres_model.nc.
  #   Type: string | string.
  input_file_path: ''
  # Format of dates contained in s3 HRES folder.
  #   Type: string.
  date_fmt: '%Y%m%d'
output_options:
  # Output Date Format for OPERA TROPO.
  #   Type: string.
  date_fmt: '%Y%m%dT%H%M%S'
  # Time the config file was created.
  #   Type: string.
  creation_time: '2025-03-16T00:14:54.417730Z'
  # Clip heights above specified maximum height.
  #   Type: integer.
  max_height: 81000
  # Output height level to hydrostatic and wet delay, default: HRES native 145 height levels.
  #   Type: array | null.
  output_heights:
    - -500.0
    - -300.0
    - -200.0
    - -100.0
    - -50.0
    - -20.0
    - -10.0
    - 0.0
    - 10.0
    - 30.96
    - 53.92
    - 79.04
    - 106.54
    - 136.62
    - 169.51
    - 205.44
    - 244.69
    - 287.52
    - 334.24
    - 385.16
    - 440.61
    - 500.95
    - 566.54
    - 637.76
    - 715.02
    - 798.72
    - 889.29
    - 987.15
    - 

In [47]:
# Print configuration parameters
tropo_workflow.model_dump()

{'input_options': {'input_file_path': 'input_data/D01010000010100001.zz.nc',
  'date_fmt': '%Y%m%d'},
 'output_options': {'date_fmt': '%Y%m%dT%H%M%S',
  'creation_time': datetime.datetime(2025, 3, 16, 0, 14, 53, 221007, tzinfo=datetime.timezone.utc),
  'max_height': 81000,
  'output_heights': [-500.0,
   -300.0,
   -200.0,
   -100.0,
   -50.0,
   -20.0,
   -10.0,
   0.0,
   10.0,
   30.96,
   53.92,
   79.04,
   106.54,
   136.62,
   169.51,
   205.44,
   244.69,
   287.52,
   334.24,
   385.16,
   440.61,
   500.95,
   566.54,
   637.76,
   715.02,
   798.72,
   889.29,
   987.15,
   1092.73,
   1206.44,
   1328.7,
   1459.91,
   1600.44,
   1750.63,
   1910.76,
   2081.09,
   2261.8,
   2452.99,
   2654.69,
   2866.83,
   3089.25,
   3321.67,
   3563.69,
   3814.82,
   4074.41,
   4341.73,
   4615.92,
   4896.02,
   5180.98,
   5469.3,
   5759.3,
   6049.89,
   6340.68,
   6631.66,
   6922.8,
   7214.09,
   7505.51,
   7797.04,
   8088.67,
   8380.36,
   8672.11,
   8963.9,
   9255.7

In [54]:
tropo_workflow.output_options.max_height

81000

In [48]:
# Get model datetime and hour
hres_dt = get_hres_datetime(tropo_workflow.input_options.input_file_path)
hres_dt

FileNotFoundError: [Errno 2] No such file or directory: '/u/aurora-r0/govorcin/01_OPERA/TROPO/interface/input_data/D01010000010100001.zz.nc'

In [24]:
# Get output filename
#OPERA_L{level}_{product_name}_{spatial_coverage}_{hres_model_datetime}_{process_start_datetime}_{WEATHER_MODEL}_{HRES_spatial_resolution}_{product_version}.nc
tropo_workflow.output_options.get_output_filename(*hres_dt)

'OPERA_L4_TROPO_GLOBAL_20190101T000000Z_20250130T025816Z_HRES_0.1_v0.1.nc'

In [25]:
#PGE runconfig contains few extra fields
pge_workflow = pge_runconfig.RunConfig()
pge_workflow = pge_workflow.from_yaml('opera_tropo/src/opera_tropo/config/default/pge_runconfig.yaml')
pge_workflow.model_dump()

{'input_file': {'input_file_path': '', 'date_fmt': '%Y%m%d'},
 'output_options': {'date_fmt': '%Y%m%dT%H%M%S',
  'creation_time': datetime.datetime(2025, 1, 30, 3, 6, 22, 296741, tzinfo=datetime.timezone.utc),
  'output_heights': [],
  'compression_kwargs': {'compression_flag': True,
   'zlib': True,
   'complevel': 5,
   'shuffle': True},
  'product_version': '0.1'},
 'primary_executable': {'product_type': 'OPERA_TROPO'},
 'product_path_group': {'product_path': PosixPath('output/historical'),
  'scratch_path': PosixPath('scratch/historical'),
  'output_directory': PosixPath('output/historical'),
  'product_version': '0.1'},
 'worker_settings': {'n_workers': 1,
  'threads_per_worker': 2,
  'max_memory': 4,
  'dask_temp_dir': 'tmp',
  'block_shape': (128, 256)},
 'log_file': PosixPath('scratch/historical/log_sas.log')}

In [29]:
# Get tropowork config from pge_config
pge_workflow.to_workflow(), pge_workflow.to_workflow().model_dump()

(TropoWorkflow(input_options=InputOptions(input_file_path='', date_fmt='%Y%m%d'), output_options=OutputOptions(date_fmt='%Y%m%dT%H%M%S', creation_time=datetime.datetime(2025, 1, 30, 3, 6, 22, 296741, tzinfo=datetime.timezone.utc), output_heights=[], compression_kwargs={'compression_flag': True, 'zlib': True, 'complevel': 5, 'shuffle': True}, product_version='0.1'), work_directory=PosixPath('/u/trappist-r0/govorcin/01_OPERA/TROPO/scratch/historical'), keep_paths_relative=False, worker_settings=WorkerSettings(n_workers=1, threads_per_worker=2, max_memory=4, dask_temp_dir='scratch/historical/tmp', block_shape=(128, 256)), log_file=PosixPath('scratch/historical/log_sas.log')),
 {'input_options': {'input_file_path': '', 'date_fmt': '%Y%m%d'},
  'output_options': {'date_fmt': '%Y%m%dT%H%M%S',
   'creation_time': datetime.datetime(2025, 1, 30, 3, 6, 22, 296741, tzinfo=datetime.timezone.utc),
   'output_heights': [],
   'compression_kwargs': {'compression_flag': True,
    'zlib': True,
    'co